In [2]:
from aocd import get_data, submit
from operator import itemgetter
import re

In [3]:
example_data = "Sensor at x=2, y=18: closest beacon is at x=-2, y=15\nSensor at x=9, y=16: closest beacon is at x=10, y=16\nSensor at x=13, y=2: closest beacon is at x=15, y=3\nSensor at x=12, y=14: closest beacon is at x=10, y=16\nSensor at x=10, y=20: closest beacon is at x=10, y=16\nSensor at x=14, y=17: closest beacon is at x=10, y=16\nSensor at x=8, y=7: closest beacon is at x=2, y=10\nSensor at x=2, y=0: closest beacon is at x=2, y=10\nSensor at x=0, y=11: closest beacon is at x=2, y=10\nSensor at x=20, y=14: closest beacon is at x=25, y=17\nSensor at x=17, y=20: closest beacon is at x=21, y=22\nSensor at x=16, y=7: closest beacon is at x=15, y=3\nSensor at x=14, y=3: closest beacon is at x=15, y=3\nSensor at x=20, y=1: closest beacon is at x=15, y=3"
real_data = get_data(day=15, year=2022)

In [4]:
pattern_text = r'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)'
pattern = re.compile(pattern_text)
pairs = [list(map(int, pattern.match(line).groups())) for line in real_data.splitlines()]

In [5]:
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1-x2) + abs(y1-y2)

In [6]:
def find_no_beacons_for_pair(pair, row, exclude_pair, minmax):
    sx, sy, bx, by = pair
    distance = manhattan_distance(sx, sy, bx, by)
    row_distance = abs(sy - row)


    if row_distance > distance:
        return []

    if row_distance == distance:
        if exclude_pair and by == row:
          return []
        else:
            if minmax and minmax[0] <= sx <= minmax[1]:
                return [sx,sx]
            else:
                return []

    interval = [sx - (distance - row_distance), sx + (distance - row_distance)]

    if exclude_pair and by == row:
      if bx < sx:
          interval = [interval[0]+1, interval[1]]
      else:
          interval =  [interval[0], interval[1]-1]

    if minmax:
        interval = [max(minmax[0], interval[0]), min(minmax[1], interval[1])]

    return interval

In [7]:
def merge(intervals):
    merged_intervals = []
    sorted_intervals = sorted(intervals, key=itemgetter(0))
    current_interval = sorted_intervals[0]
    for index in range(1, len(sorted_intervals)):
        next_interval = sorted_intervals[index]

        if current_interval[1] >= next_interval[0]:
            current_interval[1] = max(next_interval[1], current_interval[1])
        else:
            merged_intervals.append(current_interval)
            current_interval = next_interval

    merged_intervals.append(current_interval)
    return merged_intervals

def find_no_beacons(pairs, row, exclude_pair, minmax):
    intervals = []
    for pair in pairs:
        interval = find_no_beacons_for_pair(pair, row, exclude_pair, minmax)
        if len(interval) > 0:
            intervals.append(interval)

    return merge(intervals)

print(f"Number of positions in row 2000000: {sum([abs(interval[1]-interval[0]+1) for interval in find_no_beacons(pairs, 2000000, True, None)])}")

Number of positions in row 2000000: 4793062


In [ ]:
%%time
def find(max_x = 20):
    for row in range(0, max_x+1):
        intervals = find_no_beacons(pairs, row, False, (0, max_x))

        if len(intervals) > 1:
            return row, intervals[0][1]+1

        elif intervals[0][0] > 0:
            return row, intervals[0][0]

        elif intervals[0][1] < max_x:
            return row, intervals[0][1]

max_size = 4000000
x,y = find(max_size)
print(f"Tuning frequency: {y*4000000 + x}")